In [1]:
import numpy as np
import pandas as pd
import os
import multiprocessing as mul
from multiprocessing import Process
import matplotlib.pyplot as plt
import seaborn as sns

## IMPORTING AND SORTING THE DATA

WE HAVE CHOSEN ICECUBE 2008-18 DATA FOR THIS STUDY


THE ICE CUBE DATA SET HAS 1134450 NEUTRINO EVENTS


We select neutrino events with Energy >= 100TeV = 10^5 GeV

i.e log10(E/GeV) > 5


There are 192107 such neutrino events in this data

The ms pulsars are taken from the ATNF Catalogue

There are 441 pulsars (as of May 2022 when the study started)

All the pulsars lie in the declination range of -87 to +87 degrees

In [2]:
####
#### IMPORTING AND SPLITTING ICDATA $$$


path = "/media/darkwake/VIB2/Project-IceCube/icecube_10year_ps/events"
filenames = ["IC40_exp.csv", "IC59_exp.csv","IC79_exp.csv", "IC86_I_exp.csv", "IC86_II_exp.csv",
"IC86_III_exp.csv", "IC86_IV_exp.csv", "IC86_V_exp.csv", "IC86_VI_exp.csv", "IC86_VII_exp.csv"]
file = filenames[0]
f = open(os.path.join(path, file), 'r')
lines = f.readlines()
column=lines[0].split()
column.pop(0)
content = []
for file in filenames:
    f = open(os.path.join(path, file), 'r')
    lines = f.readlines()
    for line in lines[1:]:
        content.append(line.split())
    f.close()
icdata = pd.DataFrame(content, columns=column)
icdata['log10(E/GeV)'] = [float(i) for i in icdata['log10(E/GeV)']]

#icdata = icdata.sort_values('log10(E/GeV)')
#icdata = icdata.reset_index()
#icdata = icdata.drop('index', axis=1)
icdata2 = icdata[icdata['log10(E/GeV)'] > 5]
icdata2 = icdata2.reset_index()
icdata2 = icdata2.drop('index', axis=1)
icdata2

#IMPORTING MSPDATA
f = open("/media/darkwake/VIB2/Project-IceCube/10milsecpsr.txt", 'r')
lines = f.readlines()

content=[]
column=lines[3].split()
for line in lines[:]:
    content.append(line.split())
    #the INITAL DATABASE IS CLUTTERED SO WE REMOVE THE NULL COLUMNS AND OTHER CLUTTER
mspdata = pd.DataFrame(content).drop(range(0,6)).dropna().drop([2,6,8,10,11,13,14], axis=1)
f.close()
line = []
lines = []

mspdata.columns = column
column = []
content=[]
mspdata = mspdata.sort_values('DECJD')
mspdata.dropna(inplace=True)
mspdata = mspdata.reset_index()
mspdata = mspdata.drop(["index", "#"], axis=1)

uptime_path = path.replace('events', 'uptime/')
filenames = ["IC40_exp.csv", "IC59_exp.csv","IC79_exp.csv", "IC86_I_exp.csv", "IC86_II_exp.csv",
"IC86_III_exp.csv", "IC86_IV_exp.csv", "IC86_V_exp.csv", "IC86_VI_exp.csv", "IC86_VII_exp.csv"]
#filenames=["icecube_10year_ps/uptime/IC40_exp.csv", "icecube_10year_ps/uptime/IC59_exp.csv","icecube_10year_ps/uptime/IC79_exp.csv","icecube_10year_ps/uptime/IC86_I_exp.csv","icecube_10year_ps/uptime/IC86_II_exp.csv","icecube_10year_ps/uptime/IC86_III_exp.csv","icecube_10year_ps/uptime/IC86_IV_exp.csv","icecube_10year_ps/uptime/IC86_V_exp.csv","icecube_10year_ps/uptime/IC86_VI_exp.csv","icecube_10year_ps/uptime/IC86_V_exp.csv"]

file = filenames[0]
f = open(os.path.join(uptime_path, file), 'r')
lines = f.readlines()
column=lines[0].split()
column.pop(0)
content = []
f.close()
for file in filenames:
    f = open(os.path.join(uptime_path, file), 'r')
    lines = f.readlines()
    for line in lines[1:]:
        content.append(line.split())
    f.close()
icupdata = pd.DataFrame(content, columns=column)
'''icupdata = icupdata.sort_values('log10(E/GeV)')
icupdata = icupdata.reset_index()
icupdata = icupdata.drop('index', axis=1)'''

icupdata

#uptime = np.float(icupdata['MJD_stop[days]'].values) - np.float(icupdata['MJD_start[days]'].values)
tstart = [float(i) for i in icupdata['MJD_start[days]']]
tstop = [float(i) for i in icupdata['MJD_stop[days]']]
tmid = [float(i) for i in icdata['MJD[days]']]

def upt(t):
    for i in range(len(tstart)):
        if (t > tstart[i]) and (t < tstop[i]):
            #tupt.append(t - tstart[i])
            #break
            return (t - tstart[i])*86400
pool = mul.Pool()
op_async = pool.map_async(upt, tmid)
uptime = op_async.get()

len(uptime)
op_async = []
pool = []
uptime[9]

icdata.insert(6, column='uptime', value=uptime)

#icecube_10year_ps/irfs/
effa_path = path.replace('events', 'irfs')
filenames = ["IC40_exp.csv", "IC59_exp.csv","IC79_exp.csv", "IC86_I_exp.csv", "IC86_II_exp.csv"]
file = filenames[0].replace('_exp', '_effectiveArea')
f = open(os.path.join(effa_path, file), 'r')
lines = f.readlines()
column=lines[0].split()
column.pop(0)
content = []
f.close()
for file in filenames:
    file  = file.replace('_exp', '_effectiveArea')
    f = open(os.path.join(effa_path, file), 'r')
    lines = f.readlines()
    for line in lines[1:]:
        content.append(line.split())
    f.close()
iceadata = pd.DataFrame(content, columns=column)

iceadata

In [3]:
icdata2

,MJD[days],log10(E/GeV),AngErr[deg],RA[deg],Dec[deg],Azimuth[deg],Zenith[deg]
0,54562.38662244,5.17,0.62,75.024,-13.371,349.100,76.646
1,54562.39267689,5.57,1.13,305.857,-33.932,120.413,56.090
2,54562.40477593,5.80,1.11,143.516,-38.279,287.171,51.684
3,54562.42794822,5.73,0.53,310.767,-25.374,128.245,64.650
4,54562.43975209,5.38,0.64,147.017,-22.059,296.282,67.903
...,...,...,...,...,...,...,...
192102,58307.86622128,5.05,0.31,283.724,-29.301,44.569,60.727
192103,58307.87487966,5.04,0.24,301.335,-69.169,29.895,20.890
192104,58307.88476760,5.09,0.36,6.179,-68.551,328.904,21.560
192105,58307.92028692,5.23,0.66,17.387,-34.366,330.510,55.740


In [4]:
mspdata

,NAME,Gl,Gb,RAJD,DECJD,P0,F0,DIST
0,J1853-0008g,33.014,-0.453,283.30,-0.13,0.00282,354.609929,4.554
1,J1625-0021,13.890,31.827,246.2931579,-0.358044,0.0028336138772234,352.90623328676,0.951
2,J1852-0044g,32.395,-0.560,283.11,-0.73,0.00241,414.937759,4.492
3,J1835-0114,29.990,3.007,278.841325,-1.242669,0.005116387644239,195.4503977286,3.452
4,J1901-0125,32.817,-2.902,285.39083,-1.42472,0.00279,358.422939,2.360
...,...,...,...,...,...,...,...,...
436,J1944+0907,47.160,-7.357,296.03887491,9.12306353,0.005185201908798642,192.8565208431179,1.218
437,J0023+0923,111.383,-52.849,5.82032291,9.38996121,0.003050203104754390,327.8470205611185,1.818
438,B1855+09,42.290,3.060,284.401626217,9.721442158,0.005362100549682627,186.4940783438289,1.200
439,J2234+0944,76.280,-40.438,338.69522573,9.74172845,0.003627027895734199,275.7078326240928,1.587


In [5]:
max(mspdata['P0'])

'0.0098613486703673'

In [6]:
#icdata2 = nu_10gev(import_icdata())
#mspdata = import_psrdata()
msra = [float(i) for i in mspdata['RAJD']]
msdec = [float(i) for i in mspdata['DECJD']]
icra = [float(i) for i in icdata2['RA[deg]']]
icdec = [float(i) for i in icdata2['Dec[deg]']]
icang = [float(i) for i in icdata2['AngErr[deg]']]
p = len(msra)
lg = len(icra) // p + 1

In [7]:
def hvovec(lon1, lat1, lon2, lat2):

    #Convert decimal degrees to Radians:
    lon1 = np.deg2rad(lon1)
    lat1 = np.deg2rad(lat1)
    lon2 = np.deg2rad(lon2)
    lat2 = np.deg2rad(lat2)

    #Implementing Haversine Formula: 
    dlon = np.subtract(lon2, lon1)
    #dlat = np.subtract(lat2, lat1)

    a = np.add(np.multiply(np.sin(lat1), np.sin(lat2)), np.multiply(np.multiply(np.cos(lat1), np.cos(lat2)), np.cos(dlon)))

    return np.abs(np.rad2deg(np.arccos(a)))

In [8]:
msra = [float(i) for i in mspdata['RAJD']]
msdec = [float(i) for i in mspdata['DECJD']]
icra = [float(i) for i in icdata2['RA[deg]']]
icdec = [float(i) for i in icdata2['Dec[deg]']]
icang = [float(i) for i in icdata2['AngErr[deg]']]
p = len(msra)
lg = len(icra) // p + 1

In [9]:
def angfinder(b):
    ang = []
    for a in range(lg):
        
        if a != lg - 1:
        #try:
            ilo = icra[a*p:a*p + p]
            ila = icdec[a*p:a*p + p]
            lo = msra[b] * np.ones(p)
            la = msdec[b] * np.ones(p)
            temp = hvovec(ilo, ila, lo, la)
            for tt in range(len(temp)):
                if temp[tt] > 20:
                    temp[tt] = -1
            ang.extend(temp)
        else:
        #except:
            ilo = icra[a*p:]
            ila = icdec[a*p:]
            ext = len(ilo)
            lo = msra[b] * np.ones(ext)
            la = msdec[b] * np.ones(ext)
            temp = hvovec(ilo, ila, lo, la)
            #ang.extend(hvovec(ilo, ila, lo, la))
            for tt in range(len(temp)):
                if temp[tt] > 20:
                    temp[tt] = -1
            ang.extend(temp)
            
        
    return ang

pool = mul.Pool()
op_async = pool.map_async(angfinder, range(0,p))
aang = op_async.get()
op_async = []
pool = []

we only select those neutrino events, whose angular distance  is within $20^{\circ}$ of a MSP.  For a dataset of $N$ events, if $n_s$ signal events are attributed to a millisecond pulsar, the probability density of an individual event $i$ is given by:
\begin{equation}
P_i = \frac{n_s}{N} S_i + (1-\frac{n_s}{N}) B_i,
%\label{eq1}
\end{equation}
where $S_i$ and $B_i$ represent the signal and background pdfs, respectively.
The likelihood function ($\mathcal{L}$) of the entire dataset, obtained from the product of each individual PDF can be written as 
\begin{equation}
\mathcal{L} (n_s) = \prod_{i=1}^N P_i,
\end{equation}
where $P_i$ is the same as in eq1 above. The signal PDF is given by:
\begin{equation}
\large
S_i = \frac{1}{2\pi\sigma_i^2}e^{\frac{-(|\theta_i-\theta_s|)^2}{2\sigma_i^2}}
\end{equation}
where $|\theta_i-\theta_s|$ is the angular distance between the  MSP and the neutrino, whereas $\sigma_i$ is the angular uncertainty in the neutrino position.
The background PDF is determined from the average number of events per solid angle  within a declination of $\pm 3^{\circ}$ after averaging over RA. We do not incorporate  the energy information, since the pubic IceCube catalog only contains the energy of the reconstructed muon. The detection statistic used to ascertain the presence of a signal is given by:
\begin{equation}
TS (n_s) = 2 \log \frac{\mathcal{L} (n_s)}{\mathcal{L} (0)}
\end{equation}
 If the null hypothesis is true, $TS (n_s)$ obeys the $\chi^2$ distribution for one degree of freedom. We calculated $TS (n_s)$ for all MSPs in the  catalog.

### FOR $\nu_i$ and $p_j$ and  N = len(icdata2) = Total no.of $\nu$ samples
### ns[i] = no.of $\nu$ events with angles $<= 20 \deg$ with $psr_i$
### $S_i = \frac{1}{2\pi\sigma_i^2}e^{\frac{-(|\theta_i-\theta_s|)^2}{2\sigma_i^2}}$
### $|\theta_i-\theta_s|$ = aang[i][s]
### $\sigma_i$ = icang[i]           and                  sg =      $\sigma_i^2$
###

In [10]:
N = len(icdata2)

ns = []
for i in aang:
    count = 0
    for j in i:
        if j != -1:
            count += 1
    ns.append(count)
ns = np.array(ns)
    

Ns = np.sum(ns)//2

In [11]:
N

192107

n2 = []
for i in aang:
    count = 0
    for j in i:
        if j != -1:
            if j < 5:
                count += 1
    n2.append(count)

for i in aang[0]:
    if i < 5 and i!= -1:
        print(aang[0].index(i))

S_ij(10754,0)

[msra[0], msdec[0]]

[icra[10754], icdec[10754], icang[10754]]

[icra[119640], icdec[119640], icang[119640]]


S_ij(119640,0)

for i in aang[0]:
    if i < 6 and i!= -1 and i >= 5:
        print(aang[0].index(i))

S_ij(31410, 0)

for i in aang[0]:
    if i < 7 and i!= -1 and i >= 6:
        print(aang[0].index(i))

np.exp(-0.05)/150

aang[0][39160] **

S_ij(39160,0)

for i in aang[0]:
    if i < 8 and i!= -1 and i >= 7:
        print(aang[0].index(i))

S_ij(99351,0)
#aang[0][142]

ang = np.deg2rad(aang[0][99351]) ** 2

    #if ang < 20 and ang != -1:
sg = np.deg2rad(icang[99351]) ** 2
np.exp(-1 * ang / (2 * sg)) / (2 * np.pi * sg)

ang = np.deg2rad(aang[0][99351]) ** 2
err = np.deg2rad(icang[99351]) **2
np.exp(-ang/(2 * err))/(2 * np.pi * err)

np.exp(-ang/(2 * err))

-ang/(2 * err)

err

a2 = aang[0][99351] ** 2
a2

icang[99351]

In [25]:

#S_ij for ith neutrino and jth pulsar IS WRONG
#EDIT 04102022 - 11.28 - S_ij is for i^th pulsar and j^th neutrino and summed over all NEUTRINOS
def S_ij(i):
    arr = []
    
    for j in range(0,len(icdata2)):
        
        ang = aang[i][j]
        if ang != -1:
            if ang < 20:
                sg = np.deg2rad(icang[j]) ** 2
                ang = np.deg2rad(ang)
                ang = ang ** 2
                arr.append(np.exp(-ang / (2 * sg)) / (2 * np.pi * sg))
        #return
        else:
            arr.append(0)
        #return -1
    
    return np.sum(arr)
def sij():
    pool = mul.Pool()
    op_async = pool.map_async(S_ij, range(0,p))
    si = op_async.get()
    pool = []
    op_async = []
    return si
#for i in range(len(si)):

#S = Si()

In [26]:
s176 = S_ij(176)

In [27]:
s176

81013.82532461955

In [13]:
S = np.array(sij()) 

### S stores $S_i$ values for all pulsars

2103.12813

For each sample, given the Dec δi of an event, the background PDF is determined by the relative number of
events in δi ± $3^◦$ divided by the solid angle.

so calculate total no of events within delta +/- 3
and then divide by 2 pi (sin[delta+3]-sin[delta-3])

you can choose all events
withiin delta +/- 3


In [14]:
def bg(i):
    #Calculating total no of neutrino events within delta +/- 6 of ith PULSAR (not neutrino)
    count = 0
    for j in icdec:
        if abs(msdec[i] - j) <= 6:
            count+=1 
    #calculating solid angle with 3deg
    sang = 2 * np.pi * (np.sin(np.deg2rad(msdec[i] + 6)) - np.sin(np.deg2rad(msdec[i] - 6)))
    return count/sang

def Bi():
    pool = mul.Pool()
    op_async = pool.map_async(bg, range(0,p))
    return op_async.get()

B = np.array(Bi())

### B stores $B_i$ values for all pulsars

In [17]:
for i in range(len(S)):
    if S[i] == max(S):
        print(i)

176


In [18]:
max(S)

81013.82532461955

In [15]:
B

array([9.44013866e+01, 1.01254817e+02, 1.10397394e+02, 1.34020240e+02,
       1.47737784e+02, 1.85867716e+02, 7.86191589e+03, 9.28272761e+03,
       9.31811550e+03, 9.82107119e+03, 9.96038116e+03, 1.14575598e+04,
       1.28586909e+04, 1.34286636e+04, 1.50262976e+04, 1.52166228e+04,
       1.57156743e+04, 1.58616824e+04, 1.59127112e+04, 1.62574101e+04,
       1.91339314e+04, 1.92301225e+04, 2.44788793e+04, 2.48437567e+04,
       2.49859787e+04, 2.51770217e+04, 1.88918886e+02, 2.73063325e+04,
       2.73143533e+04, 2.83514279e+04, 2.85214577e+04, 2.98516568e+04,
       3.09899952e+04, 3.13816568e+04, 3.24755569e+04, 3.31511321e+04,
       3.32086031e+04, 3.41128383e+04, 3.41262352e+04, 3.44634230e+04,
       3.48159704e+04, 3.57937743e+04, 3.58048275e+04, 3.58049293e+04,
       3.58117728e+04, 3.58146264e+04, 3.58165423e+04, 3.58165423e+04,
       3.58165423e+04, 3.58165423e+04, 3.58165423e+04, 3.58165423e+04,
       3.58165423e+04, 3.58165423e+04, 3.58165423e+04, 3.58165423e+04,
      

### For $a^{th}$ pulsar

### $P_i[a] = \dfrac{ns[a]}{N} S[a] + \left(1 - \dfrac{ns[a]}{N}\right)B[a]$ if ns = no.of $\nu$ events with angles $<= 20 \deg$ with $psr_a$

                                                 OR
                                                 
### $P_i[a] = \dfrac{ns}{N} S[a] + \left(1 - \dfrac{ns}{N}\right)B[a]$     where ns is a scalar number specified manually in
### $\mathcal{L}(ns) = np.product(P_i(ns, N, S, B))$

In [55]:
for i in range(p):
    if B[i] == 0:
        print(i)

sssss = 0
for i in B:
    if i == 0:
        sssss+=1
sssss

len(B)

In [77]:
def Pr(ns, N, S, B):
    nsN = ns/N
    return np.add(np.multiply(nsN , S), np.multiply(np.subtract(1, nsN), B))

In [78]:
l = Pr(0, N, S, B)

In [79]:
np.prod(l)

/home/darkwake/.local/lib/python3.10/site-packages/numpy/core/fromnumeric.py:86: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


inf

In [81]:
max(S)

81013.82532461955

In [57]:
n2 = len(icdata2)
a = ns[0]/n2

In [58]:
a * S[0] + (1-a)*B[0]

93.7836971496685

In [59]:
[S[0], B[0]]

[6.0955712124141483e-05, 94.40138659528445]

In [60]:
1 - a

0.9934567714867235

In [61]:
0 in l

NameError: name 'l' is not defined

In [62]:
def Ls(ns1, N,S, B):
    return np.prod(Pr(ns1, N,S, B))

def L_n(S, B, N, ns):
    L = np.ones(len(ns))
    for i in range(0, len(l)):
        for j in range(len(l[i])):
            L[i] *= l[i][j]
    return L
L = L_n(S, B, N, ns)


In [63]:
def TS(ns, S, B, N):
    ts = []
    for i in ns:
        ts.append(2 * np.log10(Ls(i, N, S, B)/Ls(0, N, S, B)))
    return ts
ts = TS(ns, S, B, N)

/home/darkwake/.local/lib/python3.10/site-packages/numpy/core/fromnumeric.py:86: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
/tmp/ipykernel_26384/1122701447.py:4: RuntimeWarning: invalid value encountered in double_scalars
  ts.append(2 * np.log10(Ls(i, N, S, B)/Ls(0, N, S, B)))


In [64]:
np.array(ts)
ts[0]

nan

We use the same unbinned likelihood method as in ~\cite{Hooper,Kamionkowski,LuoZhang} which was first proposed in ~\cite{Montaruli10}. Similar to ~\cite{LuoZhang}, we only select those neutrino events, whose angular distance  is within $20^{\circ}$ of a MSP.  For a dataset of $N$ events, if $n_s$ signal events are attributed to a millisecond pulsar, the probability density of an individual event $i$ is given by:
\begin{equation}
P_i = \frac{n_s}{N} S_i + (1-\frac{n_s}{N}) B_i,
%\label{eq1}
\end{equation}
where $S_i$ and $B_i$ represent the signal and background pdfs, respectively.
The likelihood function ($\mathcal{L}$) of the entire dataset, obtained from the product of each individual PDF can be written as 
\begin{equation}
\mathcal{L} (n_s) = \prod_{i=1}^N P_i,
\end{equation}
where $P_i$ is the same as in Eq.~\ref{eq1}. The signal PDF is given by
\begin{equation}
S_i = \frac{1}{2\pi\sigma_i^2}e^{-(|\theta_i-\theta_s|)^2/2\sigma_i^2}
\end{equation}
where $|\theta_i-\theta_s|$ is the angular distance between the  MSP and the neutrino, whereas $\sigma_i$ is the angular uncertainty in the neutrino position.
The background PDF is determined from the average number of events per solid angle  within a declination of $\pm 3^{\circ}$ after averaging over RA. We do not incorporate  the energy information, since the pubic IceCube catalog only contains the energy of the reconstructed muon. The detection statistic used to ascertain the presence of a signal is given by:
\begin{equation}
TS (n_s) = 2 \log \frac{\mathcal{L} (n_s)}{\mathcal{L} (0)}
\end{equation}
 If the null hypothesis is true, $TS (n_s)$ obeys the $\chi^2$ distribution for one degree of freedom. We calculated $TS (n_s)$ for all MSPs in the  catalog.

In [ ]:
plt.figure(figsize=(8,6))
plt.yscale('log')
plt.plot(list(ts), list(ns))
plt.show()